## Package Imports

In [1]:
import numpy as np
import pandas as pd
import websocket
import json
import ssl
import threading
import sys
import pickle

## WebSocket Connection

### Methods

#### Initialization of Variables

In [4]:
#Initialize Variables
keys = []
ask_prices = []
bid_prices = []
ask_arbitrage_vals = []
bid_arbitrage_vals = []
ids = []
symbols = [["BTCEUR", "EURUSDT", "BTCUSDT"], ["ETHEUR", "EURUSDT", "ETHUSDT"], ["LTCBTC", "BTCUSDT", "LTCUSDT"], 
           ["XRPBTC", "BTCUSDT", "XRPUSDT"], ["ADABTC", "BTCUSDT", "ADAUSDT"], ["ETHBTC", "BTCEUR", "ETHEUR"],
           ["EURUSDT", "USDTGBP", "EURGBP"], ["BCHBTC", "BTCUSDT", "BCHUSDT"], ["EOSBTC", "BTCUSDT", "EOSUSDT"],
           ["LINKBTC", "BTCUSDT", "LINKUSDT"]]
data_stored = {}
for ss in symbols:
    for s in ss:
        data_stored[s] = []

#### Saver - Loader Methods

In [5]:
def save_results(keys, ask_prices, bid_prices, ids, ask_arbitrage_vals, bid_arbitrage_vals):
    with open("keys2", "wb") as k:   
        pickle.dump(keys, k) 
    
    with open("ask_prices2", "wb") as ap:   
        pickle.dump(ask_prices, ap)

    with open("bid_prices2", "wb") as bp:   
        pickle.dump(bid_prices, bp)
    
    with open("ids2", "wb") as id:   
        pickle.dump(ids, id)
    
    with open("ask_arbitrage_vals2", "wb") as aav:   
        pickle.dump(ask_arbitrage_vals, aav)
    
    with open("bid_arbitrage_vals2", "wb") as bav:   
        pickle.dump(bid_arbitrage_vals, bav)

In [4]:
def load_results(extend = False):
    loaded_keys = []
    loaded_ask_prices = []
    loaded_bid_prices = []
    loaded_ids = []
    loaded_ask_arbitrage_vals = []
    loaded_bid_arbitrage_vals = []

    open_keys = "keys"
    open_ask_prices = "ask_prices"
    open_bid_prices = "bid_prices"
    open_ids = "ids"
    open_aav = "ask_arbitrage_vals"
    open_bav = "bid_arbitrage_vals"
    if extend:
        open_keys += "2"
        open_ask_prices += "2"
        open_bid_prices += "2"
        open_ids += "2"
        open_aav += "2"
        open_bav += "2"

    with open(open_keys, "rb") as k:   
        loaded_keys = pickle.load(k) 
    
    with open(open_ask_prices, "rb") as ap:   
        loaded_ask_prices = pickle.load(ap)

    with open(open_bid_prices, "rb") as bp:   
        loaded_bid_prices = pickle.load(bp)
    
    with open(open_ids, "rb") as id:   
       loaded_ids = pickle.load(id)
    
    with open(open_aav, "rb") as aav:   
        loaded_ask_arbitrage_vals = pickle.load(aav)
    
    with open(open_bav, "rb") as bav:   
        loaded_bid_arbitrage_vals = pickle.load(bav)

    return loaded_keys, loaded_ask_prices, loaded_bid_prices, loaded_ids, loaded_ask_arbitrage_vals, loaded_bid_arbitrage_vals

#### Arbitrage Finder and Reporting Methods

In [8]:
def reporter(keys, ask_prices, bid_prices, ids, ask_arbitrage_vals, bid_arbitrage_vals):
    print(f"Number of Found Arbitrages: {len(keys)}")
    key_arbitrage_counts = np.unique(np.array(keys), return_counts=True, axis = 0)
    keys_found = key_arbitrage_counts[0]
    arbitrage_counts = key_arbitrage_counts[1]
    for i in range(len(key_arbitrage_counts[0])):
        print(f"{keys_found[i]}----> {arbitrage_counts[i]}")

    for i in range(len(keys)):
        print(f"----------{i+1}---------------")
        print("Symbols:")
        print(keys[i][0], keys[i][1], keys[i][2])
        print("IDs:")
        print(ids[i][0], ids[i][1], ids[i][2])
        print("Ask Prices:")
        print(ask_prices[i][0], ask_prices[i][1], ask_prices[i][2])
        print("Ask Arbitrage Value:")
        print(ask_arbitrage_vals[i])
        print("Bid Prices:")
        print(bid_prices[i][0], bid_prices[i][1], bid_prices[i][2])
        print("Bid Arbitrage Value:")
        print(bid_arbitrage_vals[i])
        print("---------------------------")

def check_pairs(key1, key2, key3, pairs1, pairs2, pairs3):
    global keys, ask_prices, bid_prices, ids, ask_arbitrage_vals, bid_arbitrage_vals
    last_pair1 = pairs1[-1]
    last_pair2 = pairs2[-1]
    last_pair3 = pairs3[-1]

    id1 = last_pair1[0]
    id2 = last_pair2[0]
    id3 = last_pair3[0]

    ask1 = last_pair1[1]
    ask2 = last_pair2[1]
    ask3 = last_pair3[1]

    bid1 = last_pair1[2]
    bid2 = last_pair2[2]
    bid3 = last_pair3[2]
    ask_val = ask1 * ask2 * (1/ask3) 
    bid_val = bid1 * bid2 * (1/bid3)
    if ask_val < 1 or bid_val > 1:
        if len(pairs1) > 1 and len(pairs2) > 1 and len(pairs3) > 1: #Avoid reporting the same arbitrage
            prev_pair1 = pairs1[-2]
            prev_pair2 = pairs2[-2]
            prev_pair3 = pairs3[-2]
            prev_ask1 = prev_pair1[1]
            prev_ask2 = prev_pair2[1]
            prev_ask3 = prev_pair3[1]
            prev_bid1 = prev_pair1[2]
            prev_bid2 = prev_pair2[2]
            prev_bid3 = prev_pair3[2]

            if (prev_ask1 != ask1 and prev_ask2 != ask2 and prev_ask3 != ask3) and (prev_bid1 != bid1 and prev_bid2!= bid2 and prev_bid3 != bid3):
                keys.append([key1, key2, key3])
                ids.append([id1, id2, id3])
                ask_prices.append([ask1, ask2, ask3])
                bid_prices.append([bid1, bid2, bid3])
                ask_arbitrage_vals.append(ask_val)
                bid_arbitrage_vals.append(bid_val)
        else:
            keys.append([key1, key2, key3])
            ids.append([id1, id2, id3])
            ask_prices.append([ask1, ask2, ask3])
            bid_prices.append([bid1, bid2, bid3])
            ask_arbitrage_vals.append(ask_val)
            bid_arbitrage_vals.append(bid_val)

def check_triangular_arbitrage(data_stored, symbols):
    for s in symbols:
        key1 = s[0]
        key2 = s[1]
        key3 = s[2]
        pairs1 = data_stored[key1]
        pairs2 = data_stored[key2]
        pairs3 = data_stored[key3]

        if len(pairs1) > 0 and len(pairs2) > 0 and len(pairs3) > 0:
            check_pairs(key1, key2, key3, pairs1, pairs2, pairs3)


#### Data Monitoring and Connection Methods

In [8]:
def on_message(ws, message):
    global data_stored, symbols
    # Process the data as needed
    data = json.loads(message)
    symbol = data.get("data", {}).get("s")
    update_id = data.get("data", {}).get("u")
    ask_price = data.get("data", {}).get("a")
    bid_price = data.get("data", {}).get("b")
    if update_id and symbol and ask_price and bid_price:
        data_stored[symbol].append([update_id, float(ask_price), float(bid_price)])
        check_triangular_arbitrage(data_stored, symbols)

def on_error(ws, error):
    print("ERROR")
    print(error)

def on_close(ws, close_status_code, close_msg):
    print("Closed Connection") 

def on_open(ws):
    print("Opened Connection")
    ws.send(json.dumps({"method": "SUBSCRIBE", 
                        "id": 1,
                        "params": ["btcusdt@bookTicker", "btceur@bookTicker", "eurusdt@bookTicker", 
                                   "etheur@bookTicker", "ethusdt@bookTicker", "ethbtc@bookTicker",
                                   "ltcbtc@bookTicker", "ltcusdt@bookTicker",
                                   "xrpbtc@bookTicker", "xrpusdt@bookTicker",
                                   "adabtc@booktTicker", "adausdt@bookTiker",
                                   "usdtgbp@bookTicker", "eurgbp@bookTicker",
                                   "bchbtc@bookTicker", "bchusdt@bookTicker",
                                   "eosbtc@bookTicker", "eosusdt@bookTicker",
                                   "linkbtc@bookTicker", "linkusdt@bookTicker"]}))

def close_connection(ws):
    ws.close()

### Connection

In [9]:
websocket.enableTrace(False)
socket = "wss://stream.binance.com:9443/stream"
sslopt = {"cert_reqs": ssl.CERT_NONE}
ws = websocket.WebSocketApp(socket,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close,
                                on_open= on_open)

# Schedule closing the connection after one minute
threading.Timer(5400, close_connection, args=[ws]).start()
    
ws.run_forever(sslopt=sslopt)

Opened Connection
Closed Connection


False

### Method Calls and Result Saves

In [10]:
save_results(keys, ask_prices, bid_prices, ids, ask_arbitrage_vals, bid_arbitrage_vals)

In [5]:
loaded_keys, loaded_ask_prices, loaded_bid_prices, loaded_ids, loaded_ask_arbitrage_vals, loaded_bid_arbitrage_vals = load_results()

In [6]:
loaded_keys2, loaded_ask_prices2, loaded_bid_prices2, loaded_ids2, loaded_ask_arbitrage_vals2, loaded_bid_arbitrage_vals2 = load_results(extend = True)

In [ ]:
reporter(loaded_keys2, loaded_ask_prices2, loaded_bid_prices2, loaded_ids2, loaded_ask_arbitrage_vals2, loaded_bid_arbitrage_vals2)

In [ ]:
reporter(loaded_keys, loaded_ask_prices, loaded_bid_prices, loaded_ids, loaded_ask_arbitrage_vals, loaded_bid_arbitrage_vals)